In [0]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

/kaggle/input/amazon-fine-food-reviews/hashes.txt
/kaggle/input/amazon-fine-food-reviews/database.sqlite
/kaggle/input/amazon-fine-food-reviews/Reviews.csv


In [0]:
# dependencies
import pandas as pd
from sklearn.model_selection import train_test_split
import tensorflow as tf
import numpy as np
from nltk.corpus import stopwords
stop = stopwords.words('english')

In [0]:
train = pd.read_csv('../input/amazon-fine-food-reviews/Reviews.csv')

In [0]:
train.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [0]:
train = train[['Summary', 'Text']]
train.head()

,Summary,Text
0,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
3,Cough Medicine,If you are looking for the secret ingredient i...
4,Great taffy,Great taffy at a great price. There was a wid...
7,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...


In [0]:
train['text_len'] = train['Text'].str.count(' ')
train['text_len'].describe()

count    568454.000000
mean         81.005522
std          80.807102
min           2.000000
25%          33.000000
50%          57.000000
75%          99.000000
max        3525.000000
Name: text_len, dtype: float64

we have text with a min of 2 words to max of 3525 words in text

In [0]:
train['summary_len'] = train['Summary'].str.count(' ')
train['summary_len'].describe()

count    568427.000000
mean          3.128462
std           2.619420
min           0.000000
25%           1.000000
50%           3.000000
75%           4.000000
max          41.000000
Name: summary_len, dtype: float64

here we have summaries rangeing from no words to max of 41 words also 75% of the summaries are below 4 words.
also we can say that the summaries for most of the text are small and quick summary\n
we can also see that mean of text sentences is 81 words and mean of summaries is 3 words .so very large texts have short summaries

# now let us work on short summaries and short texts so that we can build a simpler model and quick on computation

In [0]:
train = train.loc[train['summary_len']< 10]
train = train.loc[train['text_len']< 50]

In [0]:
print (train.shape)
print (train.head())

(244511, 4)
                  Summary                                               Text  \
0   Good Quality Dog Food  I have bought several of the Vitality canned d...   
1       Not as Advertised  Product arrived labeled as Jumbo Salted Peanut...   
3          Cough Medicine  If you are looking for the secret ingredient i...   
4             Great taffy  Great taffy at a great price.  There was a wid...   
7  Wonderful, tasty taffy  This taffy is so good.  It is very soft and ch...   

   text_len  summary_len  
0        48          3.0  
1        30          2.0  
3        42          1.0  
4        29          1.0  
7        27          2.0  


In [0]:
# text preprocessing - lower case letter and removing punctuation
# train['Text']
train['text_lower'] = train['Text'].str.lower()
train['text_no_punc'] = train['text_lower'].str.replace('[^\w\s]','')
#train['Summary']
train['summary_lower'] = train['Summary'].str.lower()
train['summary_no_punc'] = '_start_'+ ' ' +train['summary_lower'].str.replace('[^\w\s]','')+ ' ' +'_end_'

In [0]:
train.head()

,Summary,Text,text_len,summary_len,text_lower,text_no_punc,summary_lower,summary_no_punc
0,Good Quality Dog Food,I have bought several of the Vitality canned d...,48,3.0,i have bought several of the vitality canned d...,i have bought several of the vitality canned d...,good quality dog food,_start_ good quality dog food _end_
1,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...,30,2.0,product arrived labeled as jumbo salted peanut...,product arrived labeled as jumbo salted peanut...,not as advertised,_start_ not as advertised _end_
3,Cough Medicine,If you are looking for the secret ingredient i...,42,1.0,if you are looking for the secret ingredient i...,if you are looking for the secret ingredient i...,cough medicine,_start_ cough medicine _end_
4,Great taffy,Great taffy at a great price. There was a wid...,29,1.0,great taffy at a great price. there was a wid...,great taffy at a great price there was a wide...,great taffy,_start_ great taffy _end_
7,"Wonderful, tasty taffy",This taffy is so good. It is very soft and ch...,27,2.0,this taffy is so good. it is very soft and ch...,this taffy is so good it is very soft and che...,"wonderful, tasty taffy",_start_ wonderful tasty taffy _end_


In [0]:
max_features1 = 5000
maxlen1 = 50

max_features2 = 5000
maxlen2 = 10


In [0]:
# basic text preprocessing [ words > tokenizer > fit-on-text > text_seq > pad_seq]
tok1 = tf.keras.preprocessing.text.Tokenizer(num_words = max_features1)
tok1.fit_on_texts(list(train['text_no_punc'].astype(str)))
tf_train_text = tok1.texts_to_sequences(list(train['text_no_punc'].astype(str)))
tf_train_text = tf.keras.preprocessing.sequence.pad_sequences(tf_train_text, maxlen = maxlen1)

In [0]:
# basic text preprocessing [ words > tokenizer > fit-on-text > text_seq > pad_seq]
tok2 = tf.keras.preprocessing.text.Tokenizer(num_words = max_features2, filters = '*')
tok2.fit_on_texts(list(train['summary_no_punc'].astype(str)))
tf_train_summary = tok2.texts_to_sequences(list(train['summary_no_punc'].astype(str)))
tf_train_summary= tf.keras.preprocessing.sequence.pad_sequences(tf_train_summary, maxlen = maxlen2, padding='post')

In [0]:
# model architecture


In [0]:
print(tf_train_summary[:2,:-1])
print(tf_train_summary[:2,1:])

[[  1   4  68  28  38   2   0   0   0]
 [  1  13  45 505   2   0   0   0   0]]
[[  4  68  28  38   2   0   0   0   0]
 [ 13  45 505   2   0   0   0   0   0]]


In [0]:
vectorized_summary = tf_train_summary
# for decoder input we dont need last word as that is only for prediction
decoder_input_data = vectorized_summary[:,:-1]
# for decoder target is 1 timestep ahead from decoderinput data
decoder_target_data = vectorized_summary[:,1:]

print (f'shape of decoder input :{decoder_input_data.shape}')
print (f'shape of decoder target :{decoder_target_data.shape}')

vectorized_text = tf_train_text

encoder_input_data = vectorized_text
doc_length = encoder_input_data.shape[1]
print (f'shape of encoder input :{encoder_input_data.shape}')




shape of decoder input :(244511, 9)
shape of decoder target :(244511, 9)
shape of encoder input :(244511, 50)


In [0]:
vocab_size_encoder = len(tok1.word_index)+1
vocab_size_decoder = len(tok2.word_index)+1

In [0]:
# defining model parameters
latent_dim = 50


In [0]:
####################
####encoder model###
####################
encoder_inputs = tf.keras.Input(shape = (doc_length,), name = 'Encoder-Input')
# embedding
x=tf.keras.layers.Embedding(vocab_size_encoder,latent_dim,name='Body-word-embedding',mask_zero=False)(encoder_inputs)
# bacth normalization
x = tf.keras.layers.BatchNormalization(name = 'Encoder-batchnorm-1')(x)
# in encoder we dont need encoder output just hidden state
_, state_h = tf.keras.layers.GRU(latent_dim, return_state = True , name = 'Encoder-last-GRU')(x)
# encapsulating the encoder as a seperate so we can encode without decoding if we want to
encoder_model = tf.keras.Model(inputs = encoder_inputs , outputs = state_h, name = 'Encoder-model')

seq2seq_encoder_out = encoder_model(encoder_inputs)

####################
####decoder model###
####################
decoder_inputs = tf.keras.Input(shape = (None,) , name = 'Decoder-input')
# embedding
dec_emb=tf.keras.layers.Embedding(vocab_size_decoder,latent_dim,name='decoder-word-embedding',mask_zero=False)(decoder_inputs)
#batch normalization
dec_bn = tf.keras.layers.BatchNormalization(name = 'Decoder-batchnorm-1')(dec_emb)

dec_gru =  tf.keras.layers.GRU(latent_dim, return_state = True, return_sequences = True, name = 'Decoder-GRU')
dec_gru_output, _ = dec_gru(dec_bn, initial_state = seq2seq_encoder_out)
# batchnormalization
dec_bn_2 = tf.keras.layers.BatchNormalization(name = 'Decoder-batchnorm-2')(dec_gru_output)

# dense layers
dec_dense = tf.keras.layers.Dense(vocab_size_decoder, activation = 'softmax',name = 'Final-output-dense')
dec_outputs = dec_dense(dec_bn_2)

####################
####seq2seq model###
####################
seq2seq_model= tf.keras.Model([encoder_inputs, decoder_inputs], dec_outputs)

seq2seq_model.compile(optimizer = tf.keras.optimizers.Nadam(lr = 0.001), loss = 'sparse_categorical_crossentropy')


In [0]:
# summary of the model
seq2seq_model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Decoder-input (InputLayer)      [(None, None)]       0                                            
__________________________________________________________________________________________________
decoder-word-embedding (Embeddi (None, None, 50)     1148450     Decoder-input[0][0]              
__________________________________________________________________________________________________
Encoder-Input (InputLayer)      [(None, 50)]         0                                            
__________________________________________________________________________________________________
Decoder-batchnorm-1 (BatchNorma (None, None, 50)     200         decoder-word-embedding[0][0]     
______________________________________________________________________________________________

In [0]:
#training the model
batch_size = 64
epochs = 5
history = seq2seq_model.fit([encoder_input_data,decoder_input_data], np.expand_dims(decoder_target_data, -1)
                           ,batch_size = batch_size
                           ,epochs = epochs
                           ,validation_split = 0.1)


Train on 220059 samples, validate on 24452 samples
Epoch 1/5
220059/220059 [==============================] - 320s 1ms/sample - loss: 2.0097 - val_loss: 1.5817
Epoch 2/5
220059/220059 [==============================] - 319s 1ms/sample - loss: 1.4903 - val_loss: 1.4732
Epoch 3/5
220059/220059 [==============================] - 317s 1ms/sample - loss: 1.3772 - val_loss: 1.4304
Epoch 4/5
220059/220059 [==============================] - 317s 1ms/sample - loss: 1.3140 - val_loss: 1.4103
Epoch 5/5
220059/220059 [==============================] - 319s 1ms/sample - loss: 1.2725 - val_loss: 1.3996


In [0]:
train['text_no_punc'].tail()

568448    my only complaint is that theres so much of it...
568449    great for sesame chickenthis is a good if not ...
568450    im disappointed with the flavor the chocolate ...
568452    these are the best treats for training and rew...
568453    i am very satisfied product is as advertised i...
Name: text_no_punc, dtype: object

In [0]:
train['text_no_punc'][568452]

'these are the best treats for training and rewarding your dog for being good while grooming  lower in calories and loved by all the doggies  sweet potatoes seem to be their favorite wet noses treat'

In [0]:
test_text = ['these are the best treats for training and rewarding your dog for being good while grooming  lower in calories and loved by all the doggies  sweet potatoes seem to be their favorite wet noses treat']

In [0]:
# encoder's features for the test_text
tok1.fit_on_texts(test_text)

In [0]:
raw_tokenized = tok1.texts_to_sequences(test_text)
raw_tokenized = tf.keras.preprocessing.sequence.pad_sequences(raw_tokenized,maxlen = maxlen1)

In [0]:
# encoded state of the new sentence
body_encoding = encoder_model.predict(raw_tokenized)


In [0]:
body_encoding

array([[-0.88661003, -0.29484174,  0.8297895 ,  0.7951673 ,  0.26189852,
        -0.2639114 , -0.65622485,  0.5583709 ,  0.00414825,  0.15360205,
        -0.39446795,  0.3432643 , -0.4579382 ,  0.69474727,  0.23721723,
        -0.61368674,  0.17786449,  0.55498576,  0.06720206,  0.5624686 ,
         0.6184739 ,  0.6955943 , -0.51201224,  0.945051  , -0.63321745,
         0.8300117 ,  0.55848235,  0.9674921 ,  0.3379451 ,  0.26968428,
        -0.71685606, -0.8896725 ,  0.45976993,  0.0992775 , -0.16117108,
         0.8465392 , -0.73164713, -0.29591882,  0.20760846,  0.06894144,
        -0.2985663 ,  0.9341779 ,  0.8815483 ,  0.27380165,  0.48696768,
         0.20732337,  0.10657895,  0.17559054, -0.52167165, -0.20925842]],
      dtype=float32)

In [0]:
latent_dim = seq2seq_model.get_layer('decoder-word-embedding').output_shape[-1]


In [0]:
# Reconstruct the input into the decoder
decoder_inputs = seq2seq_model.get_layer('Decoder-input').input
dec_emb = seq2seq_model.get_layer('decoder-word-embedding')(decoder_inputs)
dec_bn = seq2seq_model.get_layer('Decoder-batchnorm-1')(dec_emb)

In [0]:
gru_inference_state_input = tf.keras.Input(shape = (latent_dim,),name = 'hidden_state_input')


In [0]:
gru_out, gru_state_out = seq2seq_model.get_layer('Decoder-GRU')([dec_bn, gru_inference_state_input])

In [0]:
# recontruct dense layers
dec_bn2 = seq2seq_model.get_layer('Decoder-batchnorm-2')(gru_out)
dense_out = seq2seq_model.get_layer('Final-output-dense')(dec_bn2)

In [0]:
decoder_model = tf.keras.Model([decoder_inputs, gru_inference_state_input],[dense_out,gru_state_out])

In [0]:
original_body_encoding = body_encoding


In [0]:
#print (np.array(tok2.word_index['_start_']))
state_value = np.array(tok2.word_index['_start_']).reshape(1,1)
state_value

array([[1]])

In [0]:
decoded_sentence = []
stop_condition = False

In [0]:
vocab_inv = dict((v, k) for k,v in tok2.word_index.items())

In [0]:
print ('sample text is %s'%test_text)
#print ('summary of test_text is',decoded_sentence)
while not stop_condition:
    preds, st = decoder_model.predict([state_value,body_encoding])
 
    pred_idx = np.argmax(preds[:,:, 2:])+2
    pred_word_str = vocab_inv[pred_idx]
    #print (pred_word_str)
    if pred_word_str == '_end_' or len(decoded_sentence) >= maxlen2:
        stop_condition = True
        break
        
    decoded_sentence.append(pred_word_str)
    
    body_encoding = st
    state_value = np.array(pred_idx).reshape(1,1)
    
summary = ' '.join(decoded_sentence)
print ('---------------------')
print ('summary of sample text is: "%s"'%summary)

sample text is ['these are the best treats for training and rewarding your dog for being good while grooming  lower in calories and loved by all the doggies  sweet potatoes seem to be their favorite wet noses treat']
---------------------
summary of sample text is: "great treats"
